In [2]:
import numpy as np
import pandas as pd
import datetime
import math

In [28]:
flow = pd.read_csv('flow.csv')
flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index(pd.DatetimeIndex(flow['date']))
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek
flow['month_no']=flow['year']+flow['month']/100
flow['hour']=flow['hour']
flow['hour_ref'] = flow.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)

flow

,hour,flow,from,to,date,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,,
2016-01-02,13,50,BOT,ZIMA,2016-01-02,2016,1,2,5,2016.01,weekend_13
2016-01-02,14,63,ZIMA,ZIM,2016-01-02,2016,1,2,5,2016.01,weekend_14
2016-01-02,15,263,ZIM,ZAMZ,2016-01-02,2016,1,2,5,2016.01,weekend_15
2016-01-02,16,100,MOZN_EDM,ZIMA,2016-01-02,2016,1,2,5,2016.01,weekend_16
2016-01-02,17,120,ZIM,ZAMZ,2016-01-02,2016,1,2,5,2016.01,weekend_17
...,...,...,...,...,...,...,...,...,...,...,...
2016-12-24,19,1,RSAS,RSAN,2016-12-24,2016,12,24,5,2016.12,weekend_19
2016-12-24,20,42,MOZN_HCB,MOZN_EDM,2016-12-24,2016,12,24,5,2016.12,weekend_20
2016-12-24,21,20,RSAS,RSAN,2016-12-24,2016,12,24,5,2016.12,weekend_21


In [29]:
flowa=flow.copy().rename(columns={'to':'from','from':'to'})
flow.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    ) 
    ,['to','from']]=flowa.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    )     
    ,['to','from']]

In [30]:
f_to=flow[['hour','flow','to','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'to': 'node'})
f_from=flow[['hour','flow','from','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'from': 'node'})
f_to['flow']=-f_to['flow']

flows=pd.concat([f_to, f_from])

flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week','month_no','hour_ref']).sum().reset_index()
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,flow
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,-160
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,-10
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,170
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,-160
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,-10
...,...,...,...,...,...,...,...,...,...,...
55861,2017-01-15,9,MOZS,2017,1,15,6,2017.01,weekend_9,-25
55862,2017-01-15,9,RSAN,2017,1,15,6,2017.01,weekend_9,50
55863,2017-01-15,9,ZAMZ,2017,1,15,6,2017.01,weekend_9,-25
55864,2017-01-15,9,ZIM,2017,1,15,6,2017.01,weekend_9,0


In [31]:
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,flow
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,-160
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,-10
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,170
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,-160
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,-10
...,...,...,...,...,...,...,...,...,...,...
55856,2017-01-15,8,RSAN,2017,1,15,6,2017.01,weekend_8,70
55857,2017-01-15,8,ZAMZ,2017,1,15,6,2017.01,weekend_8,-35
55861,2017-01-15,9,MOZS,2017,1,15,6,2017.01,weekend_9,-25
55862,2017-01-15,9,RSAN,2017,1,15,6,2017.01,weekend_9,50


In [32]:
flow_monthly=flows.groupby(['month_no','year','month','hour', 'hour_ref','node']).mean().reset_index(['hour','month','year'])
flow_monthly.rename(columns={'flow': 'flow_mean'}, inplace=True)
flow_monthly=flow_monthly.drop(['day_of_week','day'],axis=1)
flow_monthly

year  month  hour   flow_mean
month_no hour_ref   node                                   
2016.01  weekday_0  BOT       2016      1     0   17.750000
                    MOZN_HCB  2016      1     0  -93.214286
                    MOZS      2016      1     0  -14.000000
                    NAM       2016      1     0  -25.000000
                    RSAN      2016      1     0   52.000000
...                            ...    ...   ...         ...
2017.01  weekend_23 NAM       2017      1    23    4.000000
                    RSAN      2017      1    23  120.750000
                    ZAMC      2017      1    23  -37.500000
                    ZAMZ      2017      1    23  -95.000000
                    ZIMA      2017      1    23    2.000000

[4568 rows x 4 columns]

In [ ]:
flow_monthly.to_csv('dam_flow_monthly.csv')